In [1]:
import mxnet as mx
import numpy as np

/Users/marpulli/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
n_state_dims = 2
n_action_dims = 1
n_total_input_dims = n_state_dims + n_action_dims

In [3]:
X = np.random.rand(100, n_total_input_dims)
Y = np.sin(X).sum(axis=1, keepdims=True)

In [4]:
from mxfusion.common import config
config.DEFAULT_DTYPE = 'float64'

In [5]:
from mxfusion import Model, Variable
from mxfusion.components.variables import PositiveTransformation
from mxfusion.components.distributions.gp.kernels import RBF
from mxfusion.modules.gp_modules import GPRegression


m = Model()
m.N = Variable()
m.X = Variable(shape=(m.N, n_total_input_dims))
m.noise_var = Variable(shape=(1,), transformation=PositiveTransformation(), initial_value=0.01)
m.kernel = RBF(input_dim=n_total_input_dims, variance=1, lengthscale=1)
m.Y = GPRegression.define_variable(X=m.X, kernel=m.kernel, noise_var=m.noise_var, shape=(m.N, 1))

In [6]:
import mxnet as mx
from mxfusion.inference import GradBasedInference, MAP

infr = GradBasedInference(inference_algorithm=MAP(model=m, observed=[m.X, m.Y]))
infr.run(X=mx.nd.array(X, dtype='float64'), Y=mx.nd.array(Y, dtype='float64'), 
         max_iter=100, learning_rate=0.05, verbose=True)

/Users/marpulli/git/MXFusion/mxfusion/inference/inference_parameters.py:74: UserWarning: InferenceParameters has already been initialized.  The existing one will be overwritten.
  warnings.warn("InferenceParameters has already been initialized.  The existing one will be overwritten.")


Iteration 11 loss: -134.55876744817573
Iteration 21 loss: -159.54421451236675
Iteration 31 loss: -183.25152045293447
Iteration 41 loss: -206.33879771220866
Iteration 51 loss: -228.74200907343942
Iteration 61 loss: -250.52657843061718
Iteration 71 loss: -271.62110362001914
Iteration 81 loss: -291.91925795087195
Iteration 91 loss: -311.38247587935564
Iteration 100 loss: -328.3415079851717

In [7]:
from mxfusion.inference import TransferInference, ModulePredictionAlgorithm
infr_pred = TransferInference(ModulePredictionAlgorithm(model=m, observed=[m.X], target_variables=[m.Y]), 
                              infr_params=infr.params)

In [17]:
res = infr_pred.run(X=mx.nd.array(X, dtype='float64'))[0]
f_mean, f_var = res[0].asnumpy()[0], res[1].asnumpy()[0]

/Users/marpulli/git/MXFusion/mxfusion/inference/inference.py:150: UserWarning: Trying to initialize the inference twice, skipping.
  warnings.warn("Trying to initialize the inference twice, skipping.")


In [12]:
linear_policy = mx.gluon.nn.Dense(1, in_units=n_state_dims)
linear_policy.collect_params().initialize(mx.init.Normal(sigma=1.))

In [13]:
class CostFunction(mx.gluon.HybridBlock):
    def hybrid_forward(self, F, x):
        return F.abs(x)
    
cost = CostFunction()

In [14]:
n_time_steps = 100
initial_state = mx.nd.array([[-2, -2]])
intiial_action = linear_policy(initial_state)
for t in range(n_time_steps):
    mx.ndarray
    res = infr_pred.run(X=mx.nd.array(X, dtype='float64'))[0]
    f_mean, f_var = res[0].asnumpy()[0], res[1].asnumpy()[0]

/Users/marpulli/git/MXFusion/mxfusion/inference/inference.py:150: UserWarning: Trying to initialize the inference twice, skipping.
  warnings.warn("Trying to initialize the inference twice, skipping.")


In [16]:
from mxfusion.inference.model_based_alg import ModelBasedAlgorithm
ModelBasedAlgorithm(m, [m.Y], cost, linear_policy, 100, s_0, infr)

TypeError: 'GPRegressionMeanVariancePrediction' object is not callable